In [41]:
import numpy as np
from astropy.io import fits
from scipy.ndimage.filters import convolve as convolveim
from matplotlib.pylab import mpl
from code_20190825 import ikernal23 as ikernel_svm

In [42]:
# 解决画图是中文显示的问题
mpl.rcParams['font.sans-serif'] = ['SimHei']

In [43]:
ker_size = 1;
[F, Fr, Fc, Frr, Fcc, Frc] = ikernel_svm.get_par_der_2d_operator(dimension=2,ker_size=ker_size,sigam2=7,gama=100)
kernel = []
for item,item_name in zip([F, Fr, Fc, Frr, Fcc, Frc],['F', 'Fr', 'Fc', 'Frr', 'Fcc', 'Frc']):
    item_kernel = item.reshape(2 * ker_size +1, 2 * ker_size +1)
    kernel.append(item_kernel)
    np.savetxt(item_name + '.txt',item_kernel)

In [44]:
ker_size = 1
[F, Fr, Fc, Fh, Frr, Fcc, Fhh, Frc, Fch, Frh] = ikernel_svm.get_par_der_3d_operator(dimension=3,ker_size=ker_size,sigam2=7,gama=100)
kernel = []
for item in [F, Fr, Fc, Fh, Frr, Frc, Frh, Fcc, Fch, Fhh]:
    item_kernel = item.reshape(2 * ker_size +1, 2 * ker_size +1, 2 * ker_size +1)
    kernel.append(item_kernel)
#     print(item_kernel)
#     print('*'*20)

In [45]:
L = kernel[0]
Lr = kernel[1]
Lc = kernel[2]
Lh = kernel[3]
Lrr = kernel[4]
Lrc = kernel[5]
Lrh = kernel[6]
Lcc = kernel[7]
Lch = kernel[8]
Lhh = kernel[9]

In [46]:
L,Lr

(array([[[-0.05817428,  0.0148544 , -0.05817428],
         [ 0.0148544 ,  0.15199149,  0.0148544 ],
         [-0.05817428,  0.0148544 , -0.05817428]],
 
        [[ 0.0148544 ,  0.15199149,  0.0148544 ],
         [ 0.15199149,  0.37519249,  0.15199149],
         [ 0.0148544 ,  0.15199149,  0.0148544 ]],
 
        [[-0.05817428,  0.0148544 , -0.05817428],
         [ 0.0148544 ,  0.15199149,  0.0148544 ],
         [-0.05817428,  0.0148544 , -0.05817428]]]),
 array([[[ 2.94644246e-02, -1.04811993e-01,  2.94644246e-02],
         [-1.04811993e-01, -2.87534101e-01, -1.04811993e-01],
         [ 2.94644246e-02, -1.04811993e-01,  2.94644246e-02]],
 
        [[ 1.23088617e-15, -6.05598862e-16,  2.68494992e-15],
         [ 6.23315469e-15, -3.20983685e-15, -9.21343649e-16],
         [-3.21058127e-15, -2.17921448e-15,  4.03242354e-15]],
 
        [[-2.94644246e-02,  1.04811993e-01, -2.94644246e-02],
         [ 1.04811993e-01,  2.87534101e-01,  1.04811993e-01],
         [-2.94644246e-02,  1.04811993e

In [47]:
ker_size = 2
[F, Fr, Fc, Fh, Frr, Fcc, Fhh, Frc, Fch, Frh] = ikernel_svm.get_par_der_3d_operator(dimension=3,ker_size=ker_size,sigam2=7,gama=100)

In [48]:
kernel = []
for item in [Fr, Fc, Fh, Frr, Frc, Frh, Fcc, Fch, Fhh]:
    item_kernel = item.reshape(2 * ker_size +1, 2 * ker_size +1, 2 * ker_size +1)
    kernel.append(item_kernel)

In [49]:
data = fits.getdata('data/single_clump/model_1.fits')
print(data.shape) 
print(data.max())
print(np.where(data==data.max()))

(100, 100, 100)
14.247283
(array([24]), array([28]), array([16]))


In [50]:
data_convv = {}
for i, item in enumerate(['dr', 'dc', 'dh','drr', 'drc', 'drh', 'dcc', 'dch', 'dhh']):
    data_convv[item] = convolveim(data, kernel[i], mode='constant')

data_convv['drr'].shape

(100, 100, 100)

In [51]:
Drr = data_convv['drr']
Drc = data_convv['drc']
Drh = data_convv['drh']
Dcc = data_convv['dcc']
Dch = data_convv['dch']
Dhh = data_convv['dhh']

Dr = data_convv['dr']
Dc = data_convv['dc']
Dh = data_convv['dh']

In [52]:
D1 = Drr
D2 = Drr * Dcc - Drc * Drc
D3 = Drr * (Dcc * Dhh - Dch * Dch) - Drc * (Drc * Dhh - Dch * Drh) + Drh * (Drc * Dch - Drh * Dcc)


In [53]:
index_D1 = D1 < 0
index_D2 = D2 > 0
index_D3 = D3 < 0

index_Dr = np.abs(Dr) < 5
index_Dc = np.abs(Dc) < 5
index_Dh = np.abs(Dh) < 5

index_Drch = index_Dr & index_Dc & index_Dh

In [54]:
# list(set(index_D1).intersection(set(index_D2)))
points = np.where((index_D1 & index_D2 & index_D3 & index_Drch)== True)
print(points)

(array([ 6,  6,  6, ..., 42, 42, 42]), array([24, 24, 25, ..., 33, 33, 33]), array([13, 18, 12, ..., 15, 18, 20]))


In [55]:

local_points = []
for item in range(len(points[0])):
    x, y, z = points[0][item], points[1][item], points[2][item]
#     print([x,y,z,data[x,y,z]])
    local_points.append(data[x,y,z])
local_points = np.array(local_points)
print('*'*20)
print('检测得到的区域包含的点个数为：%d' %len(points[0]))

print('三维数据中的最大值为：%f' %data.max())
print('三维数据中的最大值d的下标为：%d %d %d' %np.where(data==data.max()))
print('*'*20)
print('检测区域中的最大值为：%f' %local_points.max())

********************
检测得到的区域包含的点个数为：1667
三维数据中的最大值为：14.247283
三维数据中的最大值d的下标为：24 28 16
********************
检测区域中的最大值为：14.247283


In [63]:
index_all = (index_D1 & index_D2 & index_D3 & index_Drch)
print(type(index_all))
print(index_all.astype(np.float16).max())

<class 'numpy.ndarray'>
1.0


In [64]:
noise = np.random.randn(100,100) * 1


In [66]:
from matplotlib import pyplot as plt
plt.hist(noise.reshape(1,10000),100)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([-3.82688557, -3.74655917, -3.66623277, -3.58590637, -3.50557998,
        -3.42525358, -3.34492718, -3.26460078, -3.18427439, -3.10394799,
        -3.02362159, -2.94329519, -2.86296879, -2.7826424 , -2.702316  ,
        -2.6219896 , -2.5416632 , -2.46133681, -2.38101041, -2.30068401,
        -2.22035761, -2.14003122, -2.05970482, -1.97937842, -1.89905202,
        -1.81872563, -1.73839923, -1.65807283, -1.57774643, -1.49742004,
        -1.41709364, -1.33676724, -1.25644084, -1.17611445, -1.09578805,
        -1.01546165, -0.93513525, -0.85480886, -0.77448246, -0.69415606,
        -0.61382966, -0.53350327, -0.45317687, -0.37285047, -0.29252407,
        -0.21219767, -0.13187128, -0.05154488,  0.02878152,  0.10910792,
         0.18943431, 

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


Error in callback <function flush_figures at 0x7f2b673d6f70> (for post_execute):


KeyboardInterrupt: 